In [3]:
## Import Necessary Libraries
import pprint
from fastcore.basics import patch
import networkx as nx
import matplotlib.pyplot as plt
import ipywidgets as widgets
import ipycytoscape
from ipycytoscape import CytoscapeWidget

## Creating Author Class
class Author:
    def __init__(self, first, middle, last, email=None, publications=[]):
        self.first = first
        self.middle = middle
        self.last = last
        self.email = email
        self.publications = publications

## Make format pretty
@patch
def __repr__(self:Author):
    return pprint.pformat(vars(self))

## Mock Dataset
a1 = Author('A', 'B', 'Carlson', publications=[1, 2, 3])
a2 = Author('B', 'C', 'Dawson', publications=[3, 4, 5, 6])
a3 = Author('C', 'D', 'Elfson', publications=[1, 5, 7, 8])
a4 = Author('D', 'E', 'Fitzgerald', publications=[1, 5, 9, 10])
a5 = Author('E', 'F', 'Gerard', publications=[4, 11, 12])
a5

authors = [a1, a2, a3, a4, a5]

#### Step 1: Making sure correct matches are found between authors ####

# Create the graph
G = nx.Graph()

for index_a in range(len(authors)):
    for index_b in range(index_a + 1, len(authors)):
        author_a = authors[index_a]
        author_b = authors[index_b]

        common_publications = set(author_a.publications).intersection(author_b.publications)
        if common_publications:
            # Add the weighted edge to the graph
            G.add_edge(author_a, author_b, weight=len(common_publications))
    

# Create the Cytoscape widget
cyto = CytoscapeWidget()

# Set node size based on the weight of the edges
min_weight = min(G.edges(data=True), key=lambda x: x[2]['weight'])[2]['weight']
max_weight = max(G.edges(data=True), key=lambda x: x[2]['weight'])[2]['weight']

for node, data in G.nodes(data=True):
    weight = sum(data.get('weight', 0) for _, _, data in G.edges(node, data=True))
    node_size = 30 + 20 * ((weight - min_weight) / (max_weight - min_weight))  # Adjust the size range as desired
    data['size'] = node_size

# Add nodes and edges to the Cytoscape graph
cyto.graph.add_graph_from_networkx(G)

# Create a label for the title
title_label = widgets.Label(value='Author Network Graph', style={'font-weight': 'bold'})

# Display the widgets using a VBox layout
widgets.VBox([title_label, cyto])
